# Planning observations with `astroplan`

In [0]:
import numpy as np
import pandas as pd

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
import pytz

from astroplan import Observer, FixedTarget

# Time and Dates
- ### All dates and times in are UTC: *Coordinated Universal Time* .
- All `Time` calculation assume that the time is UTC.
- UTC is related to  Greenwich Mean Time (GMT) but does not change with a change of seasons.
- Time will default to 00:00:00 UTC.

In [0]:
my_date1 = Time("2019-11-04 13:40:15")

my_date1

In [0]:
my_date1.iso

In [0]:
my_date2 = Time("2019-11-04")

my_date2.iso

### Current UTC Time

In [0]:
current_time = Time.now()    # Current UTC Time

current_time

In [0]:
print("The current time is {0.iso}".format(current_time))

### Different Date Formats

In [0]:
print("The current Julian Date is {0:.2f}".format(current_time.jd))

print("The current Modified Julian Date is {0:.2f}".format(current_time.mjd))

print("The current unix Epoch is {0:.2f}".format(current_time.unix))  # Seconds since (Jan 01, 1970 00:00:00 UTC)

print("The current fraction of a year is {0:.2f}".format(current_time.decimalyear))

### Math with Time and Dates

In [0]:
print("In 1 hour and 25 minutes it will be {0.iso} UTC".format(current_time + 1*u.h + 25*u.min))

In [0]:
Christmas = Time("2019-12-25 00:00:00")

delta_xmas = Christmas - current_time

print("It is {0:.2f} until christmas, which is {1:.2f}, or {2:.2f}"
      .format(delta_xmas.to(u.d),delta_xmas.to(u.fortnight),delta_xmas.to(u.s)))

---

### [Accurate Time](http://bmmorris.blogspot.com/2015/06/ut1-utc-and-astropy.html) - `UT1`

`AstroPy` calculates the times of events to a very high accuracy. To do this, is has to account for the fact that  Earth's rotation period is constantly changing due to tidal forces and changes in the Earth's moment of inertia.

To do this, `AstroPy` uses a time convention called `UT1`. This system is tied to the rotation of the Earth with repect to the positions of distant quasars. Since the Earth's rotation is constantly changing, the time system `UT1` is constanly changing with repect to `UTC`. 

The orientation of the Earth, which must be measured continuously to keep `UT1` accurate. This measurement is logged by the International Earth Rotation and Reference Systems Service (IERS). They publish a "bulletin" with the most recent measurements of the Earth's orientation. This bulletin is constantly being updated.

You will run into occasions when you will get a warning that your dates are out of range of the IERS bulletin. To update the bulletin, run the follow block of code:

---

In [0]:
import warnings
warnings.filterwarnings('ignore', category=Warning)

from astroplan import get_IERS_A_or_workaround

get_IERS_A_or_workaround()

# Places

## Setting your location - `Observer`

In [0]:
this_room = Observer(longitude = -122.311473 * u.deg,
                    latitude = 47 * u.deg + 39 * u.arcmin + 15 * u.arcsec,
                    elevation = 63.4 * u.m,
                    timezone = 'US/Pacific',
                    name = "Computer Lab"
                    )

In [0]:
this_room

In [0]:
this_room.name

### Working with timezones (local time)

- [Timezone List](https://en.wikipedia.org/wiki/List_of_tz_database_time_zones)
- Use the name in the **TZ database name** column.
- Only use timezone conversions for printouts, NEVER calculations!
- Working with tomezones is a [quick path](https://xkcd.com/1883/) [to maddness!](https://www.xkcd.com/1799/)

In [0]:
this_room.timezone

In [0]:
local_now = current_time.to_datetime(this_room.timezone)
print(local_now)

### Information at your location

In [0]:
sunset_here = this_room.sun_set_time(current_time, which='nearest')

print("Sunset will be at {0.iso} UTC".format(sunset_here))

In [0]:
print("Sunset will be at {0} local time".format(sunset_here.to_datetime(this_room.timezone)))

### MRO

The [Manastash Ridge Observatory (MRO)](https://sites.google.com/a/uw.edu/mro/) is operated by the Astronomy Department of the University of Washington, for the training of graduate and undergraduate students, as well as for astronomical research.

In [0]:
mro = Observer(longitude = -120.724533 * u.deg,
               latitude = 46.951089 * u.deg,
               elevation = 1198 * u.m,
               timezone = 'US/Pacific',
               name = "MRO"
              )

In [0]:
sunset_mro = mro.sun_set_time(current_time, which='nearest')

print("Sunset at MRO will be at {0} local time".format(sunset_mro.to_datetime(mro.timezone)))

In [0]:
(sunset_here - sunset_mro).to(u.min)

## When can you observe?

#### set up a reference time:

In [0]:
reference_time = current_time

### Astronomical twilight is when the Sun is 18 degrees below the horizon

* observation begin at `twilight_evening_astronomical`
* observations end at `twilight_moring_astronomical`

You can choose which `twilight_evening_astronomical` relative to the `current_time` would you like to calculate:

* `next`
* `previous`
* `nearest`

In [0]:
astro_set = mro.twilight_evening_astronomical(reference_time, which='nearest')
astro_rise = mro.twilight_morning_astronomical(reference_time, which='next')
midnight_mro = mro.midnight(reference_time, which='next')

print("Astronomical Evening Twilight starts at {0.iso} UTC".format(astro_set))
print("Astronomical Midnight is at {0.iso} UTC".format(midnight_mro))
print("Astronomical Morning Twilight starts at {0.iso} UTC".format(astro_rise))

In [0]:
observing_length = (astro_rise - astro_set).to(u.h)

print("You can observe for {0:.1f} at MRO tonight".format(observing_length))

In [0]:
# Local Times

print("Tonight's observing at MRO starts at {0},\n peaks at {1} and,\n ends at {2} local time"
      .format(astro_set.to_datetime(mro.timezone),
              midnight_mro.to_datetime(mro.timezone),
              astro_rise.to_datetime(mro.timezone)))

# Things

## Objects in the sky - `FixedTarget`

### You can define targets by [coordinates](./Astro_Coordinates.pdf)

In [0]:
coords = SkyCoord('02h19m00.0s', '+57d07m042s', frame='icrs')

ngc869 = FixedTarget(name='NGC869', coord=coords)

In [0]:
ngc869.ra

In [0]:
ngc869.ra.hms

### Can you see the object?

In [0]:
mro.target_is_up(midnight_mro, ngc869)

### Where can you see it a midnight?

In [0]:
where_to_look = mro.altaz(midnight_mro, ngc869)

In [0]:
where_to_look.alt

In [0]:
where_to_look.az

### Most targets can be defined by name

In [0]:
my_target = FixedTarget.from_name("ngc2403")

In [0]:
my_target.coord

In [0]:
my_target.ra.hms

## Objects in the sky - Moving Targets (solar system targets)

- `Astropy` used the `jplephem` package to calculate the positions
- The built-in solar system objects are: 'sun', 'mercury', 'venus', 'earth-moon-barycenter', 'earth', 'moon', 'mars', 'jupiter', 'saturn', 'uranus', 'neptune', 'pluto'

In [0]:
from astropy.coordinates import get_sun, get_body, get_moon
from astroplan import moon_illumination

### Is the Sun above the horizion now?

In [0]:
sun_now = get_body('sun',current_time)

sun_now

In [0]:
sun_location = this_room.altaz(current_time, sun_now)

sun_location.alt, sun_location.az

In [0]:
moon_now = get_body('moon',current_time)

In [0]:
moon_illumination(current_time)

In [0]:
sun_now.separation(moon_now)

# Planning - Observing at MRO

### [Air Mass](https://en.wikipedia.org/wiki/Air_mass_%28astronomy%29) is the optical path length through Earth’s atmosphere. 

* At sea-level, the air mass at the zenith is 1. 
* Air mass increases as you move toward the horizon.
* Air mass at the horizon is approximately 38.
* The best time to observe a target is at minimum airmass.
* When the airmass of your target is getting close to 2, you should be observing another target.

In [0]:
my_target = FixedTarget.from_name("ngc2403")

In [0]:
mro.target_is_up(midnight_mro, my_target)

### Object is up at midnight at MRO - good

In [0]:
altaz_my_target = mro.altaz(midnight_mro, my_target)

altaz_my_target.alt, altaz_my_target.az

### Nice high altitude - looking good

In [0]:
# You can find the airmass by using the .secz method

altaz_my_target.secz

### Airmass < 2, you are good to go.

## Planning observation is easier with plots

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

from astroplan import time_grid_from_range

from astroplan.plots import plot_airmass

#### Setup our observing window

In [0]:
start_time = astro_set
end_time = astro_rise

observing_range = [astro_set, astro_rise]

time_grid = time_grid_from_range(observing_range)

### Plot the airmass of the target over the night

In [0]:
plot_airmass(my_target, mro, time_grid);

#### This is good target for observation at MRO for most of the night

## Not all targets can (or should) be observed at all locations

In [0]:
low_target = FixedTarget.from_name("Sirius")

In [0]:
mro.target_is_up(midnight_mro, low_target)

In [0]:
plot_airmass(low_target, mro, time_grid, max_airmass=5.0);

#### Not looking good

As you can see, this is bad target for observation at MRO.

---

# List Comprehensions

List comprehensions provide a concise way to create lists (arrays). Common applications are to make new lists where each element is the result of some operations applied to each member of another sequence.

### For example: Create the list: `[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]`

In [0]:
squares = []                    # create a blank list

for x in range(10):             # foor loop 0 -> 9
    squares.append(x**2)        # calculate x**2 for each x, add to end of list

squares

### You can do the same thing with:

In [0]:
squares = [x**2 for x in range(10)]

squares

### You can include `if` statements:

In [0]:
even_squares = []

for x in range(10):

    if (x % 2 == 0):
        even_squares.append(x**2)

even_squares

### You can do the same thing with:

In [0]:
even_squares = [x**2 for x in range(10) if (x % 2 == 0)]

even_squares

---

# External list of target objects

In [0]:
target_table = pd.read_csv('./Data/ObjectList.csv')

In [0]:
target_table[0:3]

#### Shortcut to pull ALL values from a `pandas` table:

In [0]:
table_values = target_table.as_matrix()

In [0]:
table_values[0:3]

In [0]:
targets = [FixedTarget(coord=SkyCoord(ra = RA*u.hourangle, dec = DEC*u.deg), name=Name)
           for Name, RA, DEC in table_values]

In [0]:
targets[0:3]

## Observing Night

### You are the most junior member of the team, so you get stuck observing on Thanksgiving

#### Observing Window:
 * Start: `Nov 28, 2019 12:00:00 UTC`
 * End: `Nov 29, 2019 12:00:00 UTC`

In [0]:
window_start = Time("2019-11-28 12:00:00")
window_end = Time("2019-11-29 12:00:00")

### But, you get a trip to the [Canary Islands](https://en.wikipedia.org/wiki/Roque_de_los_Muchachos_Observatory)

In [0]:
#Gran Telescopio Canarias
#Observatorio Roque de los Muchachos

my_location = Observer(longitude = -17.892059 * u.deg,
                       latitude = 28.756581 * u.deg,
                       elevation = 2396 * u.m,
                       timezone = 'UTC',
                       name = "Gran Telescopio Canarias"
                      )

### What time does the window open locally?

In [0]:
print(window_start.to_datetime(my_location.timezone))

#### It is 12 pm local, so my observations start at the `nearest` astromonical twilight.

In [0]:
observe_start = my_location.twilight_evening_astronomical(window_start, which='nearest')
observe_end = my_location.twilight_morning_astronomical(window_start, which='next')

In [0]:
observing_length = (observe_end - observe_start)

print("You can observe for {0:.1f} tonight".format(observing_length.to(u.h)))

## Plot the objects - Over the observing window

In [0]:
from astroplan import time_grid_from_range
from astroplan.plots import plot_sky, plot_airmass

In [0]:
window_range = [window_start, window_end]
window_time_grid = time_grid_from_range(window_range)

In [0]:
fig,ax = plt.subplots(1,1)
fig.set_size_inches(10,5)

fig.tight_layout()

for my_object in targets:
    ax = plot_airmass(my_object, my_location, window_time_grid)

ax.vlines(observe_start.datetime, 1,3, color='r', linewidth=5)
ax.vlines(observe_end.datetime, 1,3, color='r', linewidth=5)

ax.legend(loc=0,shadow=True);

# Observing Constraints

In [0]:
from astroplan import AltitudeConstraint, AirmassConstraint, AtNightConstraint
from astroplan import observability_table

In [0]:
observing_range = [observe_start, observe_end]

In [0]:
constraints = [AirmassConstraint(2)]

In [0]:
observing_table = observability_table(constraints, my_location, targets, time_range=observing_range)

print(observing_table)

## Let us add another constraint

In [0]:
constraints.append(AltitudeConstraint(min=None, max=50*u.deg))

In [0]:
observing_table = observability_table(constraints, my_location, targets, time_range=observing_range)

print(observing_table)

## Additional Constraints

`from astroplan import CONSTRAINT`

* `AtNightConstraint()` - Constrain the Sun to be below horizon.
* `MoonIlluminationConstraint(min, max)` - Constrain the fractional illumination of the Moon.
* `MoonSeparationConstraint(min, max)` - Constrain the separation between the Moon and some targets.
* `SunSeparationConstraint(min, max)` - Constrain the separation between the Sun and some targets.

In [0]:
from astroplan import moon_illumination

In [0]:
moon_illumination(observe_start)

In [0]:
from astroplan import MoonSeparationConstraint

In [0]:
constraints.append(MoonSeparationConstraint(50*u.deg))

In [0]:
observing_table = observability_table(constraints, my_location, targets, time_range=observing_range)

print(observing_table)

In [0]:
fig,ax = plt.subplots(1,1)
fig.set_size_inches(10,5)

fig.tight_layout()

for i, my_object in enumerate(targets):

    if observing_table['ever observable'][i]:
        ax = plot_airmass(my_object, my_location, window_time_grid, max_airmass=2.0)

ax.vlines(observe_start.datetime, 1,3, color='r', linewidth=5)
ax.vlines(observe_end.datetime, 1,3, color='r', linewidth=5)

ax.legend(loc=0,shadow=True);

### Print out observing lengths for each object

In [0]:
for i, my_object in enumerate(targets):

    if observing_table['ever observable'][i]:
        name = observing_table['target name'][i]
        obj_frac = observing_table['fraction of time observable'][i]
        obj_time = obj_frac * observing_length
        output = "You can observe {0:s} for {1:.2f}".format(name, obj_time.to(u.h))
        print(output)